In [1]:
import pandas as pd
import numpy as np #to read table
from sklearn.model_selection import train_test_split #to split validation dataset
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer #to tokenize sentence
from tensorflow.keras.preprocessing.sequence import pad_sequences #to add padding
from keras.utils.np_utils import to_categorical #to make one hot vector
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense #to make 1D CNN model

In [2]:
#read table
train_data = pd.read_table('train_final.csv',sep=',')
test_data = pd.read_table('eval_final_open.csv',sep=',')

#leave only alphabet
train_data['Sentence'] = train_data['Sentence'].str.replace("[^a-zA-Z ]","")
test_data['Sentence'] = test_data['Sentence'].str.replace("[^a-zA-Z ]","")

#tokenization of train dataset and test dataset
X_train = []
for sentence in train_data['Sentence']:
    temp_X_train = word_tokenize(sentence)
    X_train.append(temp_X_train)

X_test = []
for sentence in test_data['Sentence']:
    temp_X_test = word_tokenize(sentence)
    X_test.append(temp_X_test)

vocabulary = 15000

tokenizer = Tokenizer(num_words = vocabulary)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

#assign label of train dataset
y_train = np.array(train_data['Category'])

#split train dataset and validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = len(X_train)-1000 ,shuffle = False)

#padding
padding = 35
X_train = pad_sequences(X_train, maxlen = padding)
X_val = pad_sequences(X_val, maxlen = padding)
X_test = pad_sequences(X_test, maxlen = padding)

#one hot encoding
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

<ipython-input-2-217b99d5e9ca>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['Sentence'] = train_data['Sentence'].str.replace("[^a-zA-Z ]","")
<ipython-input-2-217b99d5e9ca>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['Sentence'] = test_data['Sentence'].str.replace("[^a-zA-Z ]","")


In [3]:
#make 1D CNN model
model = Sequential()
model.add(Embedding(vocabulary, 256))
model.add(Conv1D(256, 3, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

#learning
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['acc'])
model.fit(X_train, y_train, epochs = 10, validation_data = (X_val, y_val))

Epoch 1/10
330/330 [==============================] - 9s 26ms/step - loss: 1.3988 - acc: 0.3917 - val_loss: 1.1652 - val_acc: 0.5300
Epoch 2/10
330/330 [==============================] - 8s 26ms/step - loss: 0.6691 - acc: 0.7513 - val_loss: 0.9275 - val_acc: 0.6580
Epoch 3/10
330/330 [==============================] - 8s 25ms/step - loss: 0.1208 - acc: 0.9672 - val_loss: 1.1669 - val_acc: 0.6670
Epoch 4/10
330/330 [==============================] - 8s 25ms/step - loss: 0.0211 - acc: 0.9963 - val_loss: 1.3700 - val_acc: 0.6900
Epoch 5/10
330/330 [==============================] - 8s 26ms/step - loss: 0.0051 - acc: 0.9993 - val_loss: 1.4259 - val_acc: 0.6730
Epoch 6/10
330/330 [==============================] - 8s 25ms/step - loss: 0.0027 - acc: 0.9999 - val_loss: 1.4431 - val_acc: 0.6750
Epoch 7/10
330/330 [==============================] - 8s 26ms/step - loss: 8.7109e-04 - acc: 1.0000 - val_loss: 1.4944 - val_acc: 0.6750
Epoch 8/10
330/330 [==============================] - 8s 26ms/ste

In [4]:
#make test label to submit
y_temp = model.predict(X_test)
y_test = []
for i in range(y_temp.shape[0]):
    temp_y_test = np.argmax(y_temp[i])
    y_test.append(temp_y_test)

#export as csv file
y_test = np.reshape(y_test,(-1,1))
np.savetxt('sub_label.csv',y_test,delimiter=",")